In [17]:
from enum import Enum, auto
import numpy as np
from copy import deepcopy

In [18]:
class Directions(Enum):
    N = (0, -1)
    NE = (1, -1)
    E = (1, 0)
    SE = (1, 1)
    S = (0, 1)
    SW = (-1, 1)
    W = (-1, 0)
    NW = (-1, -1)
    
class Spaces(Enum):
    EMPTY = 0
    PAWN = 1
    CASTLE = 2
    KNIGHT = 3
    BISHOP = 4
    QUEEN = 5
    KING = 6

In [19]:
class Board:
    ROWS = 8
    COLUMNS = 8
    WHITE = "white"
    BLACK = "black"
    
    def __init__(self):
        self.state = np.array([
            -2, -3, -4, -5, -6, -4, -3, -2,
            -1, -1, -1, -1, -1, -1, -1, -1,
            0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,
            1,  1,  1,  1,  1,  1,  1,  1,
            2,  3,  4,  5,  6,  4,  3,  2,
        ])
        self.current_side = self.WHITE
        self.opponent_side = self.BLACK
        self.can_castle_king_side = {
            Board.WHITE: True,
            Board.BLACK: True,
        }
        self.can_castle_queen_side = {
            Board.WHITE: True,
            Board.BLACK: True
        }
        self.en_passant = {
            Board.WHITE: None,
            Board.BLACK: None,
        }
        self.king_loc = {
            Board.WHITE: (3, self.COLUMNS - 1),
            Board.BLACK: (3, 0),
        }
        
    def __iter__(self):
        return iter(self.state)
    
    def __getitem__(self, i):
        return self.state[i]
    
    def __setitem__(self, i, val):
        self.state[i] = val
    
    def __space_to_string(self, space: int) -> str:
        if self.current_side == self.BLACK:
            space *= -1
        
        match abs(space):
            case Spaces.KING.value:
                return "♚" if space > 0 else "♔"
            case Spaces.QUEEN.value:
                return "♛" if space > 0 else "♕"
            case Spaces.BISHOP.value:
                return "♝" if space > 0 else "♗"
            case Spaces.KNIGHT.value:
                return "♞" if space > 0 else "♘"
            case Spaces.CASTLE.value: 
                return "♜" if space > 0 else "♖"
            case Spaces.PAWN.value:
                return "♟︎" if space > 0 else "♙"
            case Spaces.EMPTY.value:
                return "☐"
            case _:
                raise Exception("Invalid space")
            
    def board_to_string(self) -> str:
        returnVal = ""
        for i, space in enumerate(self):
            if i % self.ROWS == 0:
                returnVal += '\n'
            returnVal += self.__space_to_string(space)
            
        return returnVal

In [ ]:
# TODO Add castling action!
# TODO pin checking -> check during move generation
# TODO win checking -> check after opponent move
# TODO check checking -> check after opponent move
# TODO automatic queen promotion

In [22]:
class ChessEngine:
    """
    
    Numerical representation of spaces:
     - Empty space: 0
     - Pawn:        1
     - Castle:      2
     - Knight:      3
     - Bishop:      4
     - Queen:       5
     - King:        6
    The opponents pieces are multiplied by -1.
    """
    
    ROWS = 8
    COLUMNS = 8
    ACTION_SIZE = ROWS * COLUMNS * 73
    
    def __init__(self):
        self.__move_to_code = {}
        self.__code_to_move = [None] * self.ACTION_SIZE

        i = 0
        for x in range(self.COLUMNS):
            for y in range(self.ROWS):
                # Normal moves
                for direction in Directions:
                    for distance in range(1, 8):
                        self.__move_to_code[(x, y, direction, distance)] = i
                        self.__code_to_move[i] = (x, y, direction, distance)
                        i += 1
                # Knight moves
                for long_direction in (Directions.N, Directions.S):
                    for short_direction in (Directions.E, Directions.W):
                        self.__move_to_code[(x, y, "knight", long_direction, short_direction)] = i
                        self.__code_to_move[i] = (x, y, "knight", long_direction, short_direction)
                        i += 1
                for long_direction in (Directions.E, Directions.W):
                    for short_direction in (Directions.N, Directions.S):
                        self.__move_to_code[(x, y, "knight", long_direction, short_direction)] = i
                        self.__code_to_move[i] = (x, y, "knight", long_direction, short_direction)
                        i += 1
                # Underpromotion
                for direction in (Directions.NW, Directions.N, Directions.NE):
                    for promote_to in (Spaces.CASTLE, Spaces.KNIGHT, Spaces.BISHOP):
                        self.__move_to_code[(x, y, "underpromote", direction, promote_to)] = i
                        self.__code_to_move[i] = (x, y, "underpromote", direction, promote_to)
                        i += 1
        
    def start_game(self):
        return Board()
    
    def __idx_to_coords(self, idx: int) -> tuple[int]:
        return (idx % self.ROWS, idx // self.ROWS)
    
    def __coords_to_idx(self, x: int, y: int) -> int:
        return y * self.ROWS + x
    
    def __check_pin(self, board: Board, x: int, y: int, direction: Directions):
        x_diff = x - board.king_loc[0]
        y_diff = y - board.king_loc[1]
        if (x_diff != 0) and (y_diff != 0) and (x_diff != y_diff):
            return False
        
        start_distance = max(x_diff, y_diff)
        d_v = (x_diff / distance, y_diff / distance)
        if direction.value == d_v:
            return False
        
        for distance in range(start_distance + 1, self.ROWS):
            x_p, y_p = d_v[0] * distance, d_v[1] * distance
            if 0 <= x_p and x_p < self.COLUMNS and 0 <= y_p and y_p < self.COLUMNS
            
            space = board[self.__coords_to_idx(x_p, y_p)]
            if space < 0:
                return True
            if space > 0:
                return False
    
    def __get_pawn_moves(self, board: Board, x: int, y: int) -> list[int]:
        moves = []
        if y < 1:
            return moves
        
        if board[self.__coords_to_idx(x, y - 1)] == 0 and not self.__check_pin(board, x, y, Directions.N):
            moves.append(self.__move_to_code[(x, y, Directions.N, 1)])
        if board[self.__coords_to_idx(x - 1, y - 1)] < 0 and not self.__check_pin(board, x, y, Directions.NW):
            moves.append(self.__move_to_code[(x, y, Directions.NW, 1)])
        if board[self.__coords_to_idx(x + 1, y - 1)] < 0 and not self.__check_pin(board, x, y, Directions.NE):
            moves.append(self.__move_to_code[(x, y, Directions.NE, 1)])
        if y == self.ROWS - 2 and board[self.__coords_to_idx(x, y - 2)] == 0 and not self.__check_pin(board, x, y, Directions.N):
            moves.append(self.__move_to_code[(x, y, Directions.N, 2)])
        
        # En passant
        if board.en_passant[board.current_side] is not None and board.en_passant[board.current_side][1] == y - 1:            
            if board.en_passant[board.current_side][0] == x - 1 and not self.__check_pin(board, x, y, Directions.NW):
                moves.append((x, y, Directions.NW, 1))
            elif board.en_passant[board.current_side][0]== x + 1 and not self.__check_pin(board, x, y, Directions.NW):
                moves.append((x, y, Directions.NE, 1))

        return moves
    
    def __get_castle_moves(self, board: Board, x: int, y: int) -> list[int]:
        moves = []

        if self.__check_pin(board, x, y, Directions.W):      
            # Move x -> x' to the left of the castle
            for x_p in range(x - 1, 0 - 1, -1):
                x_p_space = board[self.__coords_to_idx(x_p, y)]
                if x_p_space <= 0:
                    moves.append(self.__move_to_code[(x, y, Directions.W, x - x_p)])
                if x_p_space != 0:
                    break
            # Move x -> x' to the right of the castle
            for x_p in range(x + 1, self.COLUMNS):
                x_p_space = board[self.__coords_to_idx(x_p, y)]
                if x_p_space <= 0:
                    moves.append(self.__move_to_code[(x, y, Directions.E, x_p - x)])
                if x_p_space != 0:
                    break
        if self.__check_pin(board, x, y, Directions.N):
            # Move y -> y' above the castle
            for y_p in range(y - 1, 0 - 1, -1):
                y_p_space = board[self.__coords_to_idx(x, y_p)]
                if y_p_space <= 0:
                    moves.append(self.__move_to_code[(x, y, Directions.N, y - y_p)])
                if y_p_space != 0:
                    break
            # Move y -> y' below the castle
            for y_p in range(y + 1, self.ROWS):
                y_p_space = board[self.__coords_to_idx(x, y_p)]
                if y_p_space <= 0:
                    moves.append(self.__move_to_code[(x, y, Directions.S, y_p - y)])
                if y_p_space != 0:
                    break
        
        return moves
    
    def __get_knight_moves(self, board: Board, x: int, y: int) -> list[int]:
        moves = []
        
        if y - 1 >= 0:
            if x - 2 >= 0 and board[self.__coords_to_idx(x - 2, y - 1)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.W, Directions.N)])
            if x + 2 < self.COLUMNS and board[self.__coords_to_idx(x + 2, y - 1)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.E, Directions.N)])
        if y + 1 < self.ROWS:
            if x - 2 >= 0 and board[self.__coords_to_idx(x - 2, y + 1)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.W, Directions.S)])
            if x + 2 < self.COLUMNS and board[self.__coords_to_idx(x + 2, y + 1)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.E, Directions.S)])
        if y - 2 >= 0:
            if x - 1 >= 0 and board[self.__coords_to_idx(x - 1, y - 2)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.N, Directions.W)])
            if x + 1 < self.COLUMNS and board[self.__coords_to_idx(x + 1, y - 2)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.N, Directions.E)])
        if y + 2 < self.ROWS:
            if x - 1 >= 0 and board[self.__coords_to_idx(x - 1, y + 2)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.S, Directions.W)])
            if x + 1 < self.COLUMNS and board[self.__coords_to_idx(x + 1, y + 2)] <= 0:
                moves.append(self.__move_to_code[(x, y, "knight", Directions.S, Directions.E)])
        
        return moves
    
    def __get_bishop_moves(self, board: Board, x: int, y: int) -> list[int]:
        moves = []
        
        # NW
        for x_p, y_p in zip(range(x - 1, 0 - 1, -1), range(y - 1, 0 - 1, -1)):
            new_space = board[self.__coords_to_idx(x_p, y_p)]
            if new_space <= 0:
                moves.append(self.__move_to_code[(x, y, Directions.NW, x - x_p)])
            if new_space != 0:
                break
        # NE
        for x_p, y_p in zip(range(x + 1, self.COLUMNS), range(y - 1, 0 - 1, -1)):
            new_space = board[self.__coords_to_idx(x_p, y_p)]
            if new_space <= 0:
                moves.append(self.__move_to_code[(x, y, Directions.NE, x_p - x)])
            if new_space != 0:
                break
        # SE
        for x_p, y_p in zip(range(x + 1, self.COLUMNS), range(y + 1, self.ROWS)):
            new_space = board[self.__coords_to_idx(x_p, y_p)]
            if new_space <= 0:
                moves.append(self.__move_to_code[(x, y, Directions.SE, x_p - x)])
            if new_space != 0:
                break
        # SW
        for x_p, y_p in zip(range(x - 1, 0 - 1, -1), range(y + 1, self.ROWS)):
            new_space = board[self.__coords_to_idx(x_p, y_p)]
            if new_space <= 0:
                moves.append(self.__move_to_code[(x, y, Directions.SE, x - x_p)])
            if new_space != 0:
                break
            
        return moves
    
    def __get_queen_moves(self, board: Board, x: int, y: int) -> list[int]:
        moves = self.__get_bishop_moves(board, x, y) + self.__get_castle_moves(board, x, y)
        return moves
    
    def __get_king_moves(self, board: Board, x: int, y: int) -> list[int]:
        moves = []
        
        # N
        if y > 0 and board[self.__coords_to_idx(x, y - 1)] == 0:
            moves.append(self.__move_to_code[(x, y, Directions.N, 1)])
        # NE
        if x < self.COLUMNS - 1 and y > 0 and board[self.__coords_to_idx(x + 1, y - 1)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.NE, 1)])
        # E
        if x < self.COLUMNS - 1 and board[self.__coords_to_idx(x + 1, y)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.E, 1)])
        # SE
        if x < self.COLUMNS - 1 and y < self.ROWS - 1 and board[self.__coords_to_idx(x + 1, y + 1)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.SE, 1)])
        # S
        if y < self.ROWS - 1 and board[self.__coords_to_idx(x, y + 1)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.S, 1)])
        # SW
        if x > 0 and y < self.ROWS - 1 and board[self.__coords_to_idx(x - 1, y + 1)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.SW, 1)])
        # W
        if x > 0 and board[self.__coords_to_idx(x - 1, y)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.W, 1)])
        # NW
        if x > 0 and y > 0 and board[self.__coords_to_idx(x - 1, y - 1)] < 0:
            moves.append(self.__move_to_code[(x, y, Directions.NW, 1)])

        # Castling 
        # Queen side
        if board.can_castle_queen_side[board.current_side] and np.all(
            board[self.__coords_to_idx(0, y) : self.__coords_to_idx(x, y)] == 0
        ):
            moves.append(self.__move_to_code[(x, y, Directions.W, 2)])
        # King side
        if board.can_castle_king_side[board.current_side] and np.all(
            board[self.__coords_to_idx(x + 1, y) : self.__coords_to_idx(self.ROWS, y)] == 0
        ):
            moves.append(self.__move_to_code[(x, y, Directions.E, 2)])

        return moves
    
    def get_valid_moves(self, board: Board) -> list[int]:
        moves = []
        
        for i, space in enumerate(board):
            x, y = self.__idx_to_coords(i)
            
            match space:
                case Spaces.KING.value:
                    moves.extend(self.__get_king_moves(board, x, y))
                case Spaces.QUEEN.value:
                    moves.extend(self.__get_queen_moves(board, x, y))
                case Spaces.BISHOP.value:
                    moves.extend(self.__get_bishop_moves(board, x, y))
                case Spaces.KNIGHT.value:
                    moves.extend(self.__get_knight_moves(board, x, y))
                case Spaces.CASTLE.value:
                    moves.extend(self.__get_castle_moves(board, x, y))
                case Spaces.PAWN.value:
                    moves.extend(self.__get_pawn_moves(board, x, y))
        
        return moves
                    
    def act(self, board: Board, action_code: int) -> Board:
        board = deepcopy(board)
        
        x, y, *action = self.__code_to_move[action_code]
        i = self.__coords_to_idx(x, y)
        
        if action[0] == "knight":
            long_distance = action[1]
            short_distance = action[2]
            x_p = x + 2 * long_distance.value[0] + short_distance.value[0]
            y_p = y + 2 * long_distance.value[1] + short_distance.value[1]
        
        elif action[0] == "underpromote":            
            board[i] = action[2]
        
        else:
            x_p = x + action[0].value[0] * action[1]
            y_p = y + action[0].value[1] * action[1]
            
            if board[i] == Spaces.PAWN and y == ChessEngine.ROWS - 2 and y_p == ChessEngine.ROWS - 4:
                board.en_passant[board.opponent_side] = (x, y - 1)
            if board[i] == Spaces.PAWN and board.en_passant[board.current_side] == (x_p, y_p):
                board[self.__coords_to_idx(x_p, y)] = Spaces.EMPTY
            if board[i] == Spaces.CASTLE and y == ChessEngine.ROWS - 1:
                if x == 0 :
                    board.can_castle_king_side[board.current_side] = False
                elif x == ChessEngine.COLUMNS - 1:
                    board.can_castle_queen_side[board.current_side] = False
            if board[i] == Spaces.KING and x == 4 and y == ChessEngine.ROWS - 1:
                board.can_castle_king_side[board.current_side] = False
                board.can_castle_queen_side[board.current_side] = False
            
        
        i_p = self.__coords_to_idx(x_p, y_p)
        board[i_p], board[i] = board[i], Spaces.EMPTY.value
        
        board.en_passant[board.current_side] = None
        
        return board
        
    def flip_board(self, board: Board) -> Board:
        flipped_board = deepcopy(board)
        
        flipped_board.current_side, flipped_board.opponent_side = flipped_board.opponent_side, flipped_board.current_side
        
        flipped_board.state *= -1
        flipped_board.state = np.flip(flipped_board.state)
        return flipped_board
    
    def code_to_move(self, code):
        return self.__code_to_move[code]

In [23]:
chess = ChessEngine()

state = chess.start_game()

while True:
    print(state.board_to_string())

    moves = chess.get_valid_moves(state)
    for move in moves:
        print(f"id {move}: {chess.code_to_move(move)}")

    selected_action = int(input("Move id: "))
    state = chess.act(state, selected_action)
    
    state = chess.flip_board(state)


♖♘♗♕♔♗♘♖
♙♙♙♙♙♙♙♙
☐☐☐☐☐☐☐☐
☐☐☐☐☐☐☐☐
☐☐☐☐☐☐☐☐
☐☐☐☐☐☐☐☐
♟︎♟︎♟︎♟︎♟︎♟︎♟︎♟︎
♜☐☐☐☐☐☐☐
id 438: (0, 6, <Directions.N: (0, -1)>, 1)
id 439: (0, 6, <Directions.N: (0, -1)>, 2)
id 1022: (1, 6, <Directions.N: (0, -1)>, 1)
id 1023: (1, 6, <Directions.N: (0, -1)>, 2)
id 1606: (2, 6, <Directions.N: (0, -1)>, 1)
id 1607: (2, 6, <Directions.N: (0, -1)>, 2)
id 2190: (3, 6, <Directions.N: (0, -1)>, 1)
id 2191: (3, 6, <Directions.N: (0, -1)>, 2)
id 2774: (4, 6, <Directions.N: (0, -1)>, 1)
id 2775: (4, 6, <Directions.N: (0, -1)>, 2)
id 3358: (5, 6, <Directions.N: (0, -1)>, 1)
id 3359: (5, 6, <Directions.N: (0, -1)>, 2)
id 3942: (6, 6, <Directions.N: (0, -1)>, 1)
id 3943: (6, 6, <Directions.N: (0, -1)>, 2)
id 4526: (7, 6, <Directions.N: (0, -1)>, 1)
id 4527: (7, 6, <Directions.N: (0, -1)>, 2)
id 525: (0, 7, <Directions.E: (1, 0)>, 1)
id 526: (0, 7, <Directions.E: (1, 0)>, 2)
id 527: (0, 7, <Directions.E: (1, 0)>, 3)
id 528: (0, 7, <Directions.E: (1, 0)>, 4)
id 529: (0, 7, <Directions.E: (1, 0)>, 5)
id 530:


☐☐♜☐☐☐☐☐
♟︎♟︎♟︎♟︎♟︎♟︎♟︎♟︎
☐☐☐☐☐☐☐☐
☐☐☐☐☐☐☐☐
☐☐☐☐☐☐☐☐
☐☐☐☐☐☐☐☐
♙♙♙♙♙♙♙♙
♖♘♗♔♕♗♘♖
id 438: (0, 6, <Directions.N: (0, -1)>, 1)
id 439: (0, 6, <Directions.N: (0, -1)>, 2)
id 1022: (1, 6, <Directions.N: (0, -1)>, 1)
id 1023: (1, 6, <Directions.N: (0, -1)>, 2)
id 1606: (2, 6, <Directions.N: (0, -1)>, 1)
id 1607: (2, 6, <Directions.N: (0, -1)>, 2)
id 2190: (3, 6, <Directions.N: (0, -1)>, 1)
id 2191: (3, 6, <Directions.N: (0, -1)>, 2)
id 2774: (4, 6, <Directions.N: (0, -1)>, 1)
id 2775: (4, 6, <Directions.N: (0, -1)>, 2)
id 3358: (5, 6, <Directions.N: (0, -1)>, 1)
id 3359: (5, 6, <Directions.N: (0, -1)>, 2)
id 3942: (6, 6, <Directions.N: (0, -1)>, 1)
id 3943: (6, 6, <Directions.N: (0, -1)>, 2)
id 4526: (7, 6, <Directions.N: (0, -1)>, 1)
id 4527: (7, 6, <Directions.N: (0, -1)>, 2)
id 1152: (1, 7, 'knight', <Directions.N: (0, -1)>, <Directions.W: (-1, 0)>)
id 1151: (1, 7, 'knight', <Directions.N: (0, -1)>, <Directions.E: (1, 0)>)
id 4072: (6, 7, 'knight', <Directions.N: (0, -1)>, <Directions.W: 

ValueError: invalid literal for int() with base 10: ''